In [21]:
import random

from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

import pylibgymbo as plg

random.seed(42)

In [22]:
def dump_mlp(clf, feature_vars, indent_char="", endl="\n", precision=8):
    format_str = "{:." + str(precision) + "f}"
    code = ""
    len_layers = len(clf.coefs_)
    for c, n in enumerate(feature_vars):
        code += f"{indent_char}h_0_{c} = {n};{endl}"

    for layer_id in range(len_layers):
        code += endl
        for j in range(clf.coefs_[layer_id].shape[1]):
            code += f"{indent_char}h_{layer_id + 1}_{j} = {format_str.format(clf.intercepts_[layer_id][j])}"
            for c in range(len(clf.coefs_[layer_id][:, j])):
                code += f" + ({format_str.format(clf.coefs_[layer_id][c, j])} * h_{layer_id}_{c})"
            code += f";{endl}"
            if layer_id < len_layers - 1:
                if clf.activation == "relu":
                    code += f"{indent_char}if(h_{layer_id + 1}_{j} < 0){endl}"
                    code += f"{indent_char} h_{layer_id + 1}_{j} = 0;{endl}"
            else:
                code += f"{indent_char}y_{j} = h_{layer_id + 1}_{j};{endl}"
    return code

In [34]:
X, y = make_classification(n_samples=100, random_state=1, n_features=10, n_informative=3, n_classes=3)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1)

clf = MLPClassifier(hidden_layer_sizes=(2), activation="relu", random_state=1, max_iter=100)
clf.fit(X_train, y_train)

clf.score(X_train, y_train), clf.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


(0.38666666666666666, 0.28)

In [35]:
param_low = X.min()
param_high = X.max()

num_symbolic_vars = 1
symbolic_vars_id = random.sample(list(range(X_train.shape[1])), num_symbolic_vars)

In [36]:
idx = 5
x_origin = X[idx]
y_origin = y[idx]
y_pred = clf.predict(x_origin.reshape(1, -1)).item()

feature_names = [f"var_{j}" if j in symbolic_vars_id else str(x_origin[j]) for j in range(x_origin.shape[0])]

In [37]:
mlp_code = dump_mlp(clf, feature_names)
adv_condition = "(" + " || ".join([f"(y_{c} > y_{y_pred})" for c in range(len(clf.classes_)) if y_pred != c]) + ")"
perturbation_condition = "(" + " && ".join([f"(var_{i} >= {param_low}) && (var_{i} <= {param_high})" for i in symbolic_vars_id]) + ")"

mlp_code+=f"\nif ({adv_condition} && {perturbation_condition})\n return 1;\nreturn 0;"

In [38]:
print(mlp_code)

h_0_0 = var_0;
h_0_1 = -0.04779381787454007;
h_0_2 = 2.1447272228479957;
h_0_3 = -0.7674317852237104;
h_0_4 = 1.2805020884245701;
h_0_5 = -1.654213307027733;
h_0_6 = 2.37724554987994;
h_0_7 = -0.20047204549603947;
h_0_8 = 0.24489300676725195;
h_0_9 = -1.4877177598914613;

h_1_0 = 0.51727541 + (-0.02396356 * h_0_0) + (-0.79736025 * h_0_1) + (-0.44793139 * h_0_2) + (-0.43288479 * h_0_3) + (-0.24399229 * h_0_4) + (-0.19194281 * h_0_5) + (-0.32171435 * h_0_6) + (-0.57224160 * h_0_7) + (-0.02825233 * h_0_8) + (-0.59867096 * h_0_9);
if(h_1_0 < 0)
 h_1_0 = 0;
h_1_1 = 0.58482792 + (0.23204695 * h_0_0) + (-0.18848950 * h_0_1) + (-0.49177464 * h_0_2) + (-0.12083374 * h_0_3) + (0.13853010 * h_0_4) + (0.35384015 * h_0_5) + (0.44104851 * h_0_6) + (0.16977394 * h_0_7) + (-0.00612656 * h_0_8) + (-0.33838539 * h_0_9);
if(h_1_1 < 0)
 h_1_1 = 0;

h_2_0 = -0.60121138 + (-0.50100676 * h_1_0) + (0.78164810 * h_1_1);
y_0 = h_2_0;
h_2_1 = 0.73300719 + (0.35781982 * h_1_0) + (-1.02708497 * h_1_1);
y_1 = h_2_1

In [45]:
max_depth = 65536
verbose_level = -2
num_itrs = 100
step_size = 0.01
eps = 0.000001
max_num_trials = 10
seed = 42
sign_grad = False
init_param_uniform_int = False
ignore_memory = False
use_dpll = False

In [46]:
optimizer = plg.GDOptimizer(num_itrs, step_size, eps, param_low,
            param_high, sign_grad, init_param_uniform_int, seed);

var_counter, prg = plg.gcompile(mlp_code)

In [47]:
target_pc = 0
for i, instr in enumerate(prg):
    if i > 0 and prg[i - 1].toString() == "jmp":
        print(i, instr.toString())
        target_pc = i

146 push 11
226 push 12
333 ret


In [48]:
target_pcs = {0}
plg.gexecute(prg, optimizer, target_pcs, 
             max_depth, max_num_trials,
             ignore_memory, use_dpll, verbose_level)

target_pcs = {target_pc}
constraints = plg.gexecute(prg, optimizer, target_pcs, 
                           max_depth, max_num_trials,
                           ignore_memory, use_dpll, verbose_level)
constraints

.598671)*(0-1.487718)))))+(0.781648*((((((((((0.584828+(0.232047*var_1))+((0-0.188489)*(0-0.047794)))+((0-0.491775)*var_3))+((0-0.120834)*(0-0.767432)))+(0.138530*var_5))+(0.353840*(0-1.654213)))+(0.441049*var_7))+(0.169774*(0-0.200472)))+((0-0.006127)*var_9))+((0-0.338385)*(0-1.487718))))) || ((0.733007+(0.357820*((((((((((0.517275+((0-0.023964)*var_1))+((0-0.797360)*(0-0.047794)))+((0-0.447931)*var_3))+((0-0.432885)*(0-0.767432)))+((0-0.243992)*var_5))+((0-0.191943)*(0-1.654213)))+((0-0.321714)*var_7))+((0-0.572242)*(0-0.200472)))+((0-0.028252)*var_9))+((0-0.598671)*(0-1.487718)))))+((0-1.027085)*((((((((((0.584828+(0.232047*var_1))+((0-0.188489)*(0-0.047794)))+((0-0.491775)*var_3))+((0-0.120834)*(0-0.767432)))+(0.138530*var_5))+(0.353840*(0-1.654213)))+(0.441049*var_7))+(0.169774*(0-0.200472)))+((0-0.006127)*var_9))+((0-0.338385)*(0-1.487718))))) < (((0-0.795796)+(0.912899*((((((((((0.517275+((0-0.023964)*var_1))+((0-0.797360)*(0-0.047794)))+((0-0.447931)*var_3))+((0-0.432885)*(0-0.

{'(!(((((((((((0.517275+((0-0.023964)*var_1))+((0-0.797360)*(0-0.047794)))+((0-0.447931)*var_3))+((0-0.432885)*(0-0.767432)))+((0-0.243992)*var_5))+((0-0.191943)*(0-1.654213)))+((0-0.321714)*var_7))+((0-0.572242)*(0-0.200472)))+((0-0.028252)*var_9))+((0-0.598671)*(0-1.487718))) < 0)) && (!(((((((((((0.584828+(0.232047*var_1))+((0-0.188489)*(0-0.047794)))+((0-0.491775)*var_3))+((0-0.120834)*(0-0.767432)))+(0.138530*var_5))+(0.353840*(0-1.654213)))+(0.441049*var_7))+(0.169774*(0-0.200472)))+((0-0.006127)*var_9))+((0-0.338385)*(0-1.487718))) < 0)) && (((0.733007+(0.357820*((((((((((0.517275+((0-0.023964)*var_1))+((0-0.797360)*(0-0.047794)))+((0-0.447931)*var_3))+((0-0.432885)*(0-0.767432)))+((0-0.243992)*var_5))+((0-0.191943)*(0-1.654213)))+((0-0.321714)*var_7))+((0-0.572242)*(0-0.200472)))+((0-0.028252)*var_9))+((0-0.598671)*(0-1.487718)))))+((0-1.027085)*((((((((((0.584828+(0.232047*var_1))+((0-0.188489)*(0-0.047794)))+((0-0.491775)*var_3))+((0-0.120834)*(0-0.767432)))+(0.138530*var_5))

0834)*(0-0.767432)))+(0.138530*var_5))+(0.353840*(0-1.654213)))+(0.441049*var_7))+(0.169774*(0-0.200472)))+((0-0.006127)*var_9))+((0-0.338385)*(0-1.487718))))) || ((0.733007+(0.357820*((((((((((0.517275+((0-0.023964)*var_1))+((0-0.797360)*(0-0.047794)))+((0-0.447931)*var_3))+((0-0.432885)*(0-0.767432)))+((0-0.243992)*var_5))+((0-0.191943)*(0-1.654213)))+((0-0.321714)*var_7))+((0-0.572242)*(0-0.200472)))+((0-0.028252)*var_9))+((0-0.598671)*(0-1.487718)))))+((0-1.027085)*((((((((((0.584828+(0.232047*var_1))+((0-0.188489)*(0-0.047794)))+((0-0.491775)*var_3))+((0-0.120834)*(0-0.767432)))+(0.138530*var_5))+(0.353840*(0-1.654213)))+(0.441049*var_7))+(0.169774*(0-0.200472)))+((0-0.006127)*var_9))+((0-0.338385)*(0-1.487718))))) < (((0-0.795796)+(0.912899*((((((((((0.517275+((0-0.023964)*var_1))+((0-0.797360)*(0-0.047794)))+((0-0.447931)*var_3))+((0-0.432885)*(0-0.767432)))+((0-0.243992)*var_5))+((0-0.191943)*(0-1.654213)))+((0-0.321714)*var_7))+((0-0.572242)*(0-0.200472)))+((0-0.028252)*var_9)

In [49]:
constraints

{'(!(((((((((((0.517275+((0-0.023964)*var_1))+((0-0.797360)*(0-0.047794)))+((0-0.447931)*var_3))+((0-0.432885)*(0-0.767432)))+((0-0.243992)*var_5))+((0-0.191943)*(0-1.654213)))+((0-0.321714)*var_7))+((0-0.572242)*(0-0.200472)))+((0-0.028252)*var_9))+((0-0.598671)*(0-1.487718))) < 0)) && (!(((((((((((0.584828+(0.232047*var_1))+((0-0.188489)*(0-0.047794)))+((0-0.491775)*var_3))+((0-0.120834)*(0-0.767432)))+(0.138530*var_5))+(0.353840*(0-1.654213)))+(0.441049*var_7))+(0.169774*(0-0.200472)))+((0-0.006127)*var_9))+((0-0.338385)*(0-1.487718))) < 0)) && (((0.733007+(0.357820*((((((((((0.517275+((0-0.023964)*var_1))+((0-0.797360)*(0-0.047794)))+((0-0.447931)*var_3))+((0-0.432885)*(0-0.767432)))+((0-0.243992)*var_5))+((0-0.191943)*(0-1.654213)))+((0-0.321714)*var_7))+((0-0.572242)*(0-0.200472)))+((0-0.028252)*var_9))+((0-0.598671)*(0-1.487718)))))+((0-1.027085)*((((((((((0.584828+(0.232047*var_1))+((0-0.188489)*(0-0.047794)))+((0-0.491775)*var_3))+((0-0.120834)*(0-0.767432)))+(0.138530*var_5))

In [50]:
for j in range(len(constraints)):
    x_adv = x_origin.copy()

    for i in symbolic_vars_id:
        vs = list(constraints.values())[j][1]
        if var_counter[f"var_{i}"] not in vs:
            break
        x_adv[i] = vs[var_counter[f"var_{i}"]]

    print(clf.predict_proba(x_origin.reshape(1, -1)), 
          clf.predict_proba(x_adv.reshape(1, -1)))

    print(clf.predict(x_origin.reshape(1, -1)), 
          clf.predict(x_adv.reshape(1, -1)))

1.5216740369796753
[[0.40460922 0.47389027 0.12150051]] [[0.57463775 0.3374035  0.08795875]]
[1] [0]
[[0.40460922 0.47389027 0.12150051]] [[0.40460922 0.47389027 0.12150051]]
[1] [1]
[[0.40460922 0.47389027 0.12150051]] [[0.40460922 0.47389027 0.12150051]]
[1] [1]
[[0.40460922 0.47389027 0.12150051]] [[0.40460922 0.47389027 0.12150051]]
[1] [1]
